# Unit simplification
Problems in unit simplification

In [16]:
from __future__ import print_function, division
import tellurium as te
r = te.loada("""
species S1;
S1 = 1.0;
k1 = 2.0;

k2 := k1*S1 
""")
sbml_str = r.getSBML()

""" L3V1 specificaiton p17, 39ff
The SBase abstract type is used as the base type for certain objects such as SBML, **AssignmentRule,
etc., which do not have identiers** because these components do not need to be referenced by other
components.
"""

import libsbml
doc = libsbml.readSBMLFromString(sbml_str)
model = doc.getModel()
print(model)
rule = model.getAssignmentRule('k2')
print('id:', rule.id)
print('set id:', rule.isSetId())
print('variable:', rule.variable)

# not possible to set id on rule
print('* set id of rule *')
rule.setId('my_rule')

print('id:', rule.id)
print('set id:', rule.isSetId())
print('variable:', rule.variable)

print('-'*80)

# list of all ids is not unique in model (two with same id, here k2)
all_elements = model.getListOfAllElements()

for k in range(all_elements.getSize()):
    element = all_elements.get(k)
    print(type(element), element.id)

# ListOf not iteratable
for s in model.getListOfAllElements():
    print(s.id)    

<?xml version="1.0" encoding="UTF-8"?>
<!-- Created by libAntimony version v2.8.1 on 2016-02-17 10:28 with libSBML version 5.12.1. -->
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model id="__main" name="__main">
    <listOfCompartments>
      <compartment sboTerm="SBO:0000410" id="default_compartment" spatialDimensions="3" size="1" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="S1" compartment="default_compartment" initialConcentration="1" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="k1" value="2" constant="true"/>
      <parameter id="k2" constant="false"/>
    </listOfParameters>
    <listOfRules>
      <assignmentRule variable="k2">
        <math xmlns="http://www.w3.org/1998/Math/MathML">
          <apply>
            <times/>
            <ci> k1 </ci>
            <ci> S1 </ci>
          </apply>
        </math>

TypeError: 'SBaseList' object is not iterable

In [17]:
from __future__ import print_function, division
import tellurium as te
r = te.loada("""
// Units
unit mmol = 1e-3 mole;
unit mM = mmol / litre;
unit mmol_per_s = mmol / second;
unit test = 1000 litre / metre^3

substance = mmol;
volume = litre;
area = metre^2; 
length = metre;
extent = mmol;
time_unit = second;

// Species
species S1 = 0.0

// Reaction
J0: S1 ->; v0 * k1;
v0 = 1.0 mmol_per_s;
k1 = 1.0 test;

""")
sbml_str = r.getSBML()
print(sbml_str)

<?xml version="1.0" encoding="UTF-8"?>
<!-- Created by libAntimony version v2.8.1 on 2016-02-17 10:28 with libSBML version 5.12.1. -->
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model id="__main" name="__main" substanceUnits="substance" timeUnits="time_unit" volumeUnits="volume" areaUnits="area" lengthUnits="length" extentUnits="extent">
    <listOfUnitDefinitions>
      <unitDefinition id="mmol">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="mM">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="litre" exponent="-1" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="mmol_per_s">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="second" exponent="-1" scale

In [59]:
# validate
from multiscale.sbmlutils import validation
import tempfile
import libsbml
doc = libsbml.readSBML(sbml_str)
f = tempfile.NamedTemporaryFile(suffix='.xml')
f.write(sbml_str)
f.flush()
validation.check_sbml(f.name)


 read time (ms): 0.002287
 validation error(s): 1

line 75: (10541 [Warning]) The units of the 'math' formula in a <kineticLaw> definition are expected to be the equivalent of _substance per time_.
Reference: L3V1 Sections 4.11.7, 4.2.4 AND 4.9.4
 In level 3 the expected units are extent_per_time. Expected units are mole (exponent = 1, multiplier = 1, scale = -3), second (exponent = -1, multiplier = 1, scale = 0) but the units returned by the <math> expression in the <kineticLaw> (from the <reaction> with id 'J0') are mole (exponent = 1, multiplier = 1, scale = -3), second (exponent = -1, multiplier = 1, scale = 0), litre (exponent = 1, multiplier = 1, scale = 3), metre (exponent = -3, multiplier = 1, scale = 0).



1

In [37]:
import tellurium as te
r = te.loada('''
    // Units
    m2 = meter^2;
    m3 = meter^3;
    
    // base units
    substance = mole;
    volume = m2;
    area = m3;
    length = metre;
    extent = mole;
    time_unit = second;
''')
print(r.getSBML())

<?xml version="1.0" encoding="UTF-8"?>
<!-- Created by libAntimony version v2.8.1 on 2016-02-17 12:19 with libSBML version 5.12.1. -->
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model id="__main" name="__main" substanceUnits="substance" timeUnits="time_unit" volumeUnits="volume" areaUnits="area" lengthUnits="length" extentUnits="extent">
    <listOfUnitDefinitions>
      <unitDefinition id="m2">
        <listOfUnits>
          <unit kind="metre" exponent="2" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="m3">
        <listOfUnits>
          <unit kind="metre" exponent="3" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="substance">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="volume">
        <listOfUnits>
          <unit kin

In [41]:
import tellurium as te
r = te.loada('''
    // Units
    substance = mole;
    volume = m3;
    area = m2; 
    length = metre;
    extent = mole;
    time_unit = second;
    
    unit m3 = meter^3;
    unit m2 = meter^2;
    unit mM = mole / m3;
    unit mol_per_s = mole / second;
    
    // Species
    species fru6p, atp
    fru6p = 0.1 mM;
    atp = 2.8 mM;

''')
sbml_str = r.getSBML()
print(sbml_str)

# validate
from multiscale.sbmlutils import validation
import tempfile
import libsbml
doc = libsbml.readSBML(sbml_str)
f = tempfile.NamedTemporaryFile(suffix='.xml')
f.write(sbml_str)
f.flush()
validation.check_sbml(f.name)

<?xml version="1.0" encoding="UTF-8"?>
<!-- Created by libAntimony version v2.8.1 on 2016-02-17 12:38 with libSBML version 5.12.1. -->
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model id="__main" name="__main" substanceUnits="substance" timeUnits="time_unit" volumeUnits="volume" areaUnits="area" lengthUnits="length" extentUnits="extent">
    <listOfUnitDefinitions>
      <unitDefinition id="substance">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="volume">
        <listOfUnits>
          <unit kind="metre" exponent="3" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="m3">
        <listOfUnits>
          <unit kind="metre" exponent="3" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="area">
        <listOfUnits>
          <unit k

2

In [44]:
import tellurium as te
r = te.loada('''
    // Units
    substance = mole;
    volume = m3;
    area = metre^2; 
    length = metre;
    extent = mole;
    time_unit = second;
    
    unit m3 = meter^3;
    unit mM = mole / m3;
    unit mol_per_s = mole / second;
    
    // Species
    fru6p = 0.1 mM;
    atp = 2.8 mM;

    // Parameters and rule
    PFK2_n_native = 1.3 dimensionless;       
    PFK2_n_phospho = 2.1 dimensionless;
    PFK2_k_fru6p_native = 0.016 mM; 
    PFK2_k_fru6p_phospho = 0.050 mM;
    PFK2_k_atp_native = 0.28 mM;
    PFK2_k_atp_phospho = 0.65 mM;
    PFK2_Vmax = 0.0042 mol_per_s;

    PFK2_native := PFK2_Vmax * fru6p^PFK2_n_native / (fru6p^PFK2_n_native + PFK2_k_fru6p_native^PFK2_n_native) * atp/(atp + PFK2_k_atp_native);
    PFK2_native has mol_per_s;
''')
sbml_str = r.getSBML()
print(sbml_str)

# validate
from multiscale.sbmlutils import validation
import tempfile
import libsbml
doc = libsbml.readSBML(sbml_str)
f = tempfile.NamedTemporaryFile(suffix='.xml')
f.write(sbml_str)
f.flush()
validation.check_sbml(f.name)

<?xml version="1.0" encoding="UTF-8"?>
<!-- Created by libAntimony version v2.8.1 on 2016-02-17 12:48 with libSBML version 5.12.1. -->
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model id="__main" name="__main" substanceUnits="substance" timeUnits="time_unit" volumeUnits="volume" areaUnits="area" lengthUnits="length" extentUnits="extent">
    <listOfUnitDefinitions>
      <unitDefinition id="substance">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="volume">
        <listOfUnits>
          <unit kind="metre" exponent="3" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="m3">
        <listOfUnits>
          <unit kind="metre" exponent="3" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="area">
        <listOfUnits>
          <unit k

4